<a href="https://colab.research.google.com/github/claredavies/InterviewTaskDataCleaning/blob/main/InterviewDataPrepTask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Libraries

In [47]:
import numpy as np
from google.colab import files
import pandas as pd
from matplotlib import pyplot as plt
from pandas.api.types import is_numeric_dtype
from pandas.api.types import is_string_dtype
import re
import socket
import datetime
from datetime import datetime

In [48]:
pd.set_option('max_rows', 99999)
pd.set_option('max_colwidth', 400)
pd.set_option('display.max_rows', None)

# Read in files

In [3]:
!git clone https://github.com/claredavies/InterviewTaskDataCleaning.git

Cloning into 'InterviewTaskDataCleaning'...
remote: Enumerating objects: 19, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 19 (delta 6), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (19/19), done.


In [4]:
%cd InterviewTaskDataCleaning

/content/InterviewTaskDataCleaning


In [5]:
df = pd.read_csv('InputData.csv')
df.head(3)

,Ref,First_Name,Last_Name,Email,Gender,IP_Address,Date_of_Birth
0,1,Germayne,Simmons,gsimmons0@cdbaby.com,Male,24.48.223.251,18/11/1959
1,2,Cosmo,Pleass,cpleass1@netscape.com,Male,108.87.136.28,08/11/1935
2,3,Henrik,Boag,hboag2@bizjournals.com,Male,221.105.60.101,07/10/1989


# Functions

In [6]:
def boolCheckLengthOverLimitString(value, limit):
  lengthFound = len(value)
  if(lengthFound > limit):
    return True
  else:
    return False

In [7]:
def boolCheckLengthOverLimitInt(value, limit):
  stringValue = str(value)
  return boolCheckLengthOverLimitString(stringValue, limit)

In [8]:
def checkCaseCorrect(string):
  lengthWord = len(string)
  if string[0].isupper() == False:
    return False
  for i in range(1, lengthWord):
    if string[i].isupper() == True:
      return False
  return True

In [9]:
def checkOnlyCharacter(string):
  lengthWord = len(string)
  if string.isalpha() == False:
    for i in range(0, lengthWord):
      if string[i].isalpha() == False:
        if(string[i].find("-") == -1 or string[i].find("'") == -1):
          return False
    return True
  else:
    return True

In [10]:
def checkIfSpacesBetweenWords(string):
  res = bool(re.search(r"\s", string))
  return res

In [11]:
def checkMaleOrFemale(string):
  lowercaseString = string.lower()
  if lowercaseString == 'female':
    return True
  elif lowercaseString == 'male':
    return True
  else:
    return False

In [12]:
def checkIfIPV4Valid(string):
  try:
      socket.inet_aton(string)
      return True
      # legal
  except socket.error:
    return False

In [13]:
def checkFormatDate(string):
  format = '%d/%m/%Y'
  try:
    datetime.strptime(string, format)
    return True
  except ValueError:
    return False 

In [14]:
def check18OrOverBy1stDec2020(startDate):
  endDate = '01/12/2020'
  endDate = datetime.strptime(endDate, '%d/%m/%Y')
  startDate = datetime.strptime(startDate, '%d/%m/%Y')
  yearsOld = endDate.year - startDate.year - ((endDate.month, endDate.day) < (startDate.month, startDate.day))
  if yearsOld >= 18:
    return True
  else:
    return False

# Check Duplicates

In [15]:
dfWithoutRef = df.drop('Ref', axis=1)
dfWithoutRef.head(3)

,First_Name,Last_Name,Email,Gender,IP_Address,Date_of_Birth
0,Germayne,Simmons,gsimmons0@cdbaby.com,Male,24.48.223.251,18/11/1959
1,Cosmo,Pleass,cpleass1@netscape.com,Male,108.87.136.28,08/11/1935
2,Henrik,Boag,hboag2@bizjournals.com,Male,221.105.60.101,07/10/1989


In [17]:
duplicate = df[dfWithoutRef.duplicated()]
print("Duplicate Rows :")
print(duplicate)

Duplicate Rows :
Empty DataFrame
Columns: [Ref, First_Name, Last_Name, Email, Gender, IP_Address, Date_of_Birth]
Index: []


# Check size - should be 1000

In [46]:
print(df.shape)

(1000, 7)


# Check Ref Column

In [20]:
# check numeric
print(is_numeric_dtype(df['Ref']))

# check all values unique
print(df['Ref'].is_unique)

True
True


In [21]:
# want to add in check for unique
def checkRefColumn(input, columnName, index):
  dfFlaws = pd.DataFrame(columns=['RowIndex', 'ColumnName', 'ColumnValue', 'ColumnIssue','Fixed',"FixedResult"])

  if boolCheckLengthOverLimitInt(input, 8) == True:
    issueMessage = 'bool check length greater than 8'
    dfFlaws = dfFlaws.append({'RowIndex': index,'ColumnName': columnName, 'ColumnValue': input, 'ColumnIssue': issueMessage, 'Fixed': False, 'FixedResult': 'N/A'}, ignore_index=True)

  elif pd.isna(input):
    issueMessage = 'value is null'
    dfFlaws = dfFlaws.append({'RowIndex': index, 'ColumnName': columnName, 'ColumnValue': input, 'ColumnIssue': issueMessage, 'Fixed': False, 'FixedResult': 'N/A'}, ignore_index=True)

  return dfFlaws

# Check First Name

Check type (character only)

In [22]:
# check numeric
print(is_string_dtype(df['First_Name']))

True


In [23]:
def checkColumnFirstName(input, columnName, index):
  dfFlaws = pd.DataFrame(columns=['RowIndex', 'ColumnName', 'ColumnValue', 'ColumnIssue','Fixed',"FixedResult"])

  if pd.isna(input):
    issueMessage = 'value is null'
    dfFlaws = dfFlaws.append({'RowIndex': index,'ColumnName': columnName, 'ColumnValue': input, 'ColumnIssue': issueMessage, 'Fixed': False, 'FixedResult': 'N/A'}, ignore_index=True)

  else:
    if boolCheckLengthOverLimitInt(input, 20) == True:
      issueMessage = 'value is over length'
      dfFlaws = dfFlaws.append({'RowIndex': index,'ColumnName': columnName, 'ColumnValue': input, 'ColumnIssue': issueMessage, 'Fixed': False, 'FixedResult': 'N/A'}, ignore_index=True)


    if(checkCaseCorrect(input) == False):
      issueMessage = 'case is incorrect'
      dfFlaws = dfFlaws.append({'RowIndex': index,'ColumnName': columnName, 'ColumnValue': input, 'ColumnIssue': issueMessage, 'Fixed': False, 'FixedResult': 'N/A'}, ignore_index=True)


    # need to search for numbers within
    if checkOnlyCharacter(input) == False:
      issueMessage = 'not only characters'
      dfFlaws = dfFlaws.append({'RowIndex': index,'ColumnName': columnName, 'ColumnValue': input, 'ColumnIssue': issueMessage, 'Fixed': False, 'FixedResult': 'N/A'}, ignore_index=True)

  return dfFlaws

# Check Second Name

In [24]:
# check numeric
print(is_string_dtype(df['Last_Name']))

True


In [38]:
def checkColumnSecondName(input, columnName, index):
  dfFlaws = pd.DataFrame(columns=['RowIndex', 'ColumnName', 'ColumnValue', 'ColumnIssue','Fixed',"FixedResult"])

  if pd.isna(input):
    issueMessage = 'value is null'
    dfFlaws = dfFlaws.append({'RowIndex': index,'ColumnName': columnName, 'ColumnValue': input, 'ColumnIssue': issueMessage, 'Fixed': False, 'FixedResult': 'N/A'}, ignore_index=True)

  else:
    if boolCheckLengthOverLimitInt(input, 40) == True:
      issueMessage = 'value is over length'
      dfFlaws = dfFlaws.append({'RowIndex': index,'ColumnName': columnName, 'ColumnValue': input, 'ColumnIssue': issueMessage, 'Fixed': False, 'FixedResult': 'N/A'}, ignore_index=True)

    if(checkCaseCorrect(input) == False):
      issueMessage = 'case is incorrect'
      dfFlaws = dfFlaws.append({'RowIndex': index,'ColumnName': columnName, 'ColumnValue': input, 'ColumnIssue': issueMessage, 'Fixed': False, 'FixedResult': 'N/A'}, ignore_index=True)


    # need to search for numbers within
    if checkOnlyCharacter(input) == False:
      issueMessage = 'not only characters'
      dfFlaws = dfFlaws.append({'RowIndex': index,'ColumnName': columnName, 'ColumnValue': input, 'ColumnIssue': issueMessage, 'Fixed': False, 'FixedResult': 'N/A'}, ignore_index=True)

    if checkIfSpacesBetweenWords(input) == True:
      issueMessage = 'space between words'
      dfFlaws = dfFlaws.append({'RowIndex': index,'ColumnName': columnName, 'ColumnValue': input, 'ColumnIssue': issueMessage, 'Fixed': False, 'FixedResult': 'N/A'}, ignore_index=True)
  
  return dfFlaws

# Check Email 

In [26]:
# check string
print(is_string_dtype(df['Email']))

True


In [27]:
def checkColumnEmail(input, columnName, index):
  dfFlaws = pd.DataFrame(columns=['RowIndex', 'ColumnName', 'ColumnValue', 'ColumnIssue','Fixed',"FixedResult"])

  if boolCheckLengthOverLimitInt(input, 254) == True:
    issueMessage = 'value is over length'
    dfFlaws = dfFlaws.append({'RowIndex': index,'ColumnName': columnName, 'ColumnValue': input, 'ColumnIssue': issueMessage, 'Fixed': False, 'FixedResult': 'N/A'}, ignore_index=True)

  return dfFlaws

# Check Gender

In [39]:
def checkColumnGender(input, columnName, index):
  dfFlaws = pd.DataFrame(columns=['RowIndex', 'ColumnName', 'ColumnValue', 'ColumnIssue','Fixed',"FixedResult"])

  if pd.notna(input):
    if type(input) == str:
      if boolCheckLengthOverLimitInt(input, 6) == True:
        issueMessage = 'value is over length'
        dfFlaws = dfFlaws.append({'RowIndex': index,'ColumnName': columnName, 'ColumnValue': input, 'ColumnIssue': issueMessage, 'Fixed': False, 'FixedResult': 'N/A'}, ignore_index=True)

      if checkMaleOrFemale(input) == False:
        issueMessage = 'not male or female'
        dfFlaws = dfFlaws.append({'RowIndex': index,'ColumnName': columnName, 'ColumnValue': input, 'ColumnIssue': issueMessage, 'Fixed': False, 'FixedResult': 'N/A'}, ignore_index=True)
    else:
      issueMessage = 'not string'
      dfFlaws = dfFlaws.append({'RowIndex': index,'ColumnName': columnName, 'ColumnValue': input, 'ColumnIssue': issueMessage, 'Fixed': False, 'FixedResult': 'N/A'}, ignore_index=True)
  else:
      issueMessage = 'value is null'
      dfFlaws = dfFlaws.append({'RowIndex': index,'ColumnName': columnName, 'ColumnValue': input, 'ColumnIssue': issueMessage, 'Fixed': False, 'FixedResult': 'N/A'}, ignore_index=True)
  
  return dfFlaws

# Check IP Address

In [40]:
def checkColumnIPAddress(input, columnName, index):
  dfFlaws = pd.DataFrame(columns=['RowIndex', 'ColumnName', 'ColumnValue', 'ColumnIssue','Fixed',"FixedResult"])

  if pd.notna(row['IP_Address']):
    if type(row['IP_Address']) == str:

      if boolCheckLengthOverLimitInt(row['IP_Address'], 15) == True:
        issueMessage = 'value is over length'
        dfFlaws = dfFlaws.append({'RowIndex': index,'ColumnName': columnName, 'ColumnValue': input, 'ColumnIssue': issueMessage, 'Fixed': False, 'FixedResult': 'N/A'}, ignore_index=True)
      
      if checkIfIPV4Valid(row['IP_Address']) == False:
        issueMessage = 'not valid IPV4'
        dfFlaws = dfFlaws.append({'RowIndex': index,'ColumnName': columnName, 'ColumnValue': input, 'ColumnIssue': issueMessage, 'Fixed': False, 'FixedResult': 'N/A'}, ignore_index=True)

    else:
      issueMessage = 'not string'
      dfFlaws = dfFlaws.append({'RowIndex': index,'ColumnName': columnName, 'ColumnValue': input, 'ColumnIssue': issueMessage, 'Fixed': False, 'FixedResult': 'N/A'}, ignore_index=True)

  else:
    issueMessage = 'value is null'
    dfFlaws = dfFlaws.append({'RowIndex': index,'ColumnName': columnName, 'ColumnValue': input, 'ColumnIssue': issueMessage, 'Fixed': False, 'FixedResult': 'N/A'}, ignore_index=True)
  
  return dfFlaws

# Check Date of Birth

In [41]:
def checkColumnDateOfBirth(input, columnName, index):
  dfFlaws = pd.DataFrame(columns=['RowIndex', 'ColumnName', 'ColumnValue', 'ColumnIssue','Fixed',"FixedResult"])

  if pd.notna(row['Date_of_Birth']):
    if type(row['Date_of_Birth']) == str:

      if boolCheckLengthOverLimitInt(row['Date_of_Birth'], 10) == True:
        issueMessage = 'value is over length'
        dfFlaws = dfFlaws.append({'RowIndex': index,'ColumnName': columnName, 'ColumnValue': input, 'ColumnIssue': issueMessage, 'Fixed': False, 'FixedResult': 'N/A'}, ignore_index=True)
      
      if checkFormatDate(row['Date_of_Birth']) == False:
        issueMessage = 'date of birth format no dd/mm/yyyy'
        dfFlaws = dfFlaws.append({'RowIndex': index,'ColumnName': columnName, 'ColumnValue': input, 'ColumnIssue': issueMessage, 'Fixed': False, 'FixedResult': 'N/A'}, ignore_index=True)
      
      else:
        if check18OrOverBy1stDec2020(row['Date_of_Birth']) == False:
           issueMessage = 'Issue with age - not 18 by 1st December 2020'
           dfFlaws = dfFlaws.append({'RowIndex': index,'ColumnName': columnName, 'ColumnValue': input, 'ColumnIssue': issueMessage, 'Fixed': False, 'FixedResult': 'N/A'}, ignore_index=True)
    else:
        issueMessage = 'not string'
        dfFlaws = dfFlaws.append({'RowIndex': index,'ColumnName': columnName, 'ColumnValue': input, 'ColumnIssue': issueMessage, 'Fixed': False, 'FixedResult': 'N/A'}, ignore_index=True)

  else:
    issueMessage = 'value is null'
    dfFlaws = dfFlaws.append({'RowIndex': index,'ColumnName': columnName, 'ColumnValue': input, 'ColumnIssue': issueMessage, 'Fixed': False, 'FixedResult': 'N/A'}, ignore_index=True)
  
  return dfFlaws

# Overall

In [45]:
# check each row
dfFlaws = pd.DataFrame(columns=['RowIndex', 'ColumnName', 'ColumnValue', 'ColumnIssue','Fixed',"FixedResult"])

for index, row in df.iterrows():
  dfFlawsRefColumn = checkRefColumn(row["Ref"], "Ref", index)
  if dfFlawsRefColumn.empty == False:
    dfFlaws = dfFlaws.append(dfFlawsRefColumn, ignore_index=True)

  dfFlawsFirstNameColumn = checkColumnFirstName(row["First_Name"], "First_Name", index)
  if dfFlawsFirstNameColumn.empty == False:
    dfFlaws = dfFlaws.append(dfFlawsFirstNameColumn, ignore_index=True)

  dfFlawsLastNameColumn = checkColumnSecondName(row["Last_Name"], "Last_Name", index)
  if dfFlawsLastNameColumn.empty == False:
    dfFlaws = dfFlaws.append(dfFlawsLastNameColumn, ignore_index=True)

  dfFlawsEmailColumn = checkColumnEmail(row["Email"], "Email", index)
  if dfFlawsEmailColumn.empty == False:
    dfFlaws = dfFlaws.append(dfFlawsEmailColumn, ignore_index=True)

  dfFlawsGenderColumn = checkColumnGender(row["Gender"], "Gender", index)
  if dfFlawsGenderColumn.empty == False:
    dfFlaws = dfFlaws.append(dfFlawsGenderColumn, ignore_index=True)

  dfFlawsIPAddressColumn = checkColumnIPAddress(row["IP_Address"], "IP_Address", index)
  if dfFlawsIPAddressColumn.empty == False:
    dfFlaws = dfFlaws.append(dfFlawsIPAddressColumn, ignore_index=True)

  dfFlawsDateOfBirthColumn = checkColumnDateOfBirth(row["Date_of_Birth"], "Date_of_Birth", index)
  if dfFlawsDateOfBirthColumn.empty == False:
    dfFlaws = dfFlaws.append(dfFlawsDateOfBirthColumn, ignore_index=True)

In [49]:
print(dfFlaws.head())

  RowIndex  ColumnName                              ColumnValue  \
0        5  First_Name                               christabel   
1        5   Last_Name                                    start   
2        8  IP_Address  1df6:b3b9:cdb7:d246:3fe9:7288:46ee:e528   
3        8  IP_Address  1df6:b3b9:cdb7:d246:3fe9:7288:46ee:e528   
4       26   Last_Name                                      NaN   

            ColumnIssue  Fixed FixedResult  
0     case is incorrect  False         N/A  
1     case is incorrect  False         N/A  
2  value is over length  False         N/A  
3        not valid IPV4  False         N/A  
4         value is null  False         N/A  


In [51]:
dfFlaws.to_csv("ProblemsWithData", sep='\t', encoding='utf-8')